In [ ]:
import csv
import copy
import math
import ctypes
import pickle
import random
import numpy as np
import open3d as o3d

In [ ]:
libNDT6D = np.ctypeslib.load_library(libname="ndt_registration_knn", 
                                        loader_path="/home/onepiece/Downloads/ndt-registration-generalized/build/")

In [ ]:
class NDT6DRegistration():
    def __init__(self, src_map, trg_map):
        libNDT6D.NDTRegistration_map.argtypes = [np.ctypeslib.ndpointer(dtype=np.float64, ndim=trg_map.ndim, shape=trg_map.shape,
                                                 flags='C_CONTIGUOUS'), ctypes.c_size_t,
                                                 np.ctypeslib.ndpointer(dtype=np.float64, ndim=src_map.ndim, shape=src_map.shape,
                                                 flags='C_CONTIGUOUS'), ctypes.c_size_t]
        libNDT6D.NDTRegistration_map.restype = ctypes.c_void_p
        
        libNDT6D.NDTRegistration_d2d.argtypes = [ctypes.c_void_p, np.ctypeslib.ndpointer(dtype=np.float64, ndim=2, shape=(4, 4), flags='C_CONTIGUOUS')]
        libNDT6D.NDTRegistration_d2d.restype = ctypes.c_double

        self.obj = libNDT6D.NDTRegistration_map(trg_map, len(trg_map), src_map, len(src_map))
    
    def d2d(self):
        res = np.eye(4)
        score = libNDT6D.NDTRegistration_d2d(self.obj, res)
        return score, res

In [ ]:
folder = "/home/onepiece/Dataset/Greenhouse/Vineyard/bag_prependicular/"

In [ ]:
def rgb2lab ( inputColor ) :
    ### RGB should be float with RGB/255, RGB is numpy array N*3
    RGB = np.zeros_like(inputColor)
    ind1 = np.argwhere(inputColor>0.04045)
    ind2 = np.argwhere(inputColor<=0.04045)

    RGB[ind1[:,0], ind1[:,1]] = ((inputColor[ind1[:,0], ind1[:,1]]+0.055)/1.055)**2.4
    RGB[ind2[:,0], ind2[:,1]] = inputColor[ind2[:,0], ind2[:,1]]/12.92
    RGB = RGB*100

    XYZ = np.zeros_like(inputColor)

    XYZ[:, 0] = RGB [:,0] * 0.4124 + RGB [:,1] * 0.3576 + RGB [:,2] * 0.1805
    XYZ[:, 1] = RGB [:,0] * 0.2126 + RGB [:,1] * 0.7152 + RGB [:,2] * 0.0722
    XYZ[:, 2] = RGB [:,0] * 0.0193 + RGB [:,1] * 0.1192 + RGB [:,2] * 0.9505

    XYZ[:, 0] = XYZ[:, 0] / 95.047         # ref_X =  95.047   Observer= 2°, Illuminant= D65
    XYZ[:, 1] = XYZ[:, 1] / 100.0          # ref_Y = 100.000
    XYZ[:, 2] = XYZ[:, 2] / 108.883        # ref_Z = 108.883

    ind1 = np.argwhere(XYZ>0.008856)
    ind2 = np.argwhere(XYZ<=0.008856)
    XYZ[ind1[:,0], ind1[:,1]] = XYZ[ind1[:,0], ind1[:,1]]**(1.0/3.0)
    XYZ[ind2[:,0], ind2[:,1]] = (7.787*XYZ[ind2[:,0], ind2[:,1]]) + (16 / 116)

    Lab = np.zeros_like(inputColor)
    Lab[:, 0] = ( 116 * XYZ[:, 1] ) - 16
    Lab[:, 1] = 500 * ( XYZ[:, 0] - XYZ[:, 1] )
    Lab[:, 2] = 200 * ( XYZ[:, 1] - XYZ[:, 2] )
    return Lab

In [ ]:
def load_point_cloud(file_index, voxel_size=0.0025, normalize=False):
    ### read rgbd files and cereate point cloud
    color_raw = o3d.io.read_image(folder+"color/"+str(file_index)+".jpg")
    if normalize:
        img = np.asarray(color_raw)
        img_sum = np.sum(img, axis=2)
        img_normal = np.ones_like(img)
        img_normal[:,:,0] = 255*(img[:,:,0]/img_sum)
        img_normal[:,:,1] = 255*(img[:,:,1]/img_sum)
        img_normal[:,:,2] = 255*(img[:,:,2]/img_sum)
        color_raw = o3d.geometry.Image(img_normal)
    depth_raw = o3d.io.read_image(folder+"depth/"+str(file_index)+".png")
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, 
                                                                        depth_scale=800.0,
                                                                        convert_rgb_to_intensity=False)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image,
                    o3d.camera.PinholeCameraIntrinsic(width = 1280, height = 720, 
                                                      fx = 924.616, fy = 924.584, 
                                                      cx = 651.648, cy = 355.379))

    
    cl, ind = pcd.remove_radius_outlier(nb_points=20, radius=0.01)
    inlier_cloud = pcd.select_by_index(ind)
    inlier_cloud = inlier_cloud.voxel_down_sample(voxel_size=voxel_size)

    return inlier_cloud

In [ ]:
class NDT6dCell():
    """
    NDT cell type object.
    """
    def __init__(self):
        self.mean_ = []
        self.cov_ = []
        self.minN_ = 15 # min number of points required
        self.has_gaussian_ = False
        self.points = []
        self.clusters = 0
    
    def addPoint(self, pnt):
        """
        Adding point to the list. input should be a Numpy vector
        """
        self.points.append(pnt)
    
    def calculateGaussian(self):
        """
        Calculate mean covarience and inverse covarience for the cell
        """        
        if (len(self.points) < self.minN_):
            return False
        
        self.mean_ = np.mean(self.points, axis=0)
        diff = self.points - self.mean_
        self.cov_ = (diff.T@diff)/(len(diff)-1)
        if (np.linalg.det(self.cov_) < 0):
            return False
        try:
            icov = np.linalg.inv(self.cov_)
        except np.linalg.LinAlgError:
            self.has_gaussian_ = False
            return False
        self.has_gaussian_ = True
    
    def hasGaussian(self):
        return self.has_gaussian_
        
    def getCovarience(self):
        return self.cov_
    
    def getMean(self):
        return self.mean_

In [ ]:
class HashGrid6d(object):
    """
    HashGrid3d is a a spatial index which can be used for creating 3D NDT maps.
    """
    def __init__(self, cell_size):
        self.cell_size = cell_size
        self.grid = {}


    def key(self, point):
        """
        Key for dictionary. returns a tuple.
        """
        pnt = np.floor(point[:3]/self.cell_size)
        return (int(pnt[0]), int(pnt[1]), int(pnt[2]))
    
    def getCell(self, pnt):
        """
        Returns the cell at input point, if cell  
        """
        key_vector = self.key(pnt)
        if (key_vector not in self.grid):
            self.grid[key_vector] = NDT6dCell()
        return self.grid.get(key_vector)
    
    def hasCell(self, pnt):
        """
        Check if a cell is present at the enquiry point
        """
        key_vector = self.key(pnt)
        if (key_vector not in self.grid):
            return False
        return True
    
    def getGrid(self):
        return self.grid

In [ ]:
class NDTMap():
    def __init__(self, resolution=0.5):
        self.cell_size = resolution
        self.map_ = HashGrid6d(self.cell_size)
        self.num_of_gaussians = 0
    
    def addPoint(self, pnt):
        """
        Adding point to the the map. Input is numpy vector 6
        """
        self.map_.getCell(pnt).addPoint(pnt)
        
    def addPointCloud(self, pnts):
        """
        Adding point cloud to the the map. Input is list of numpy vector n*6
        """
        for pnt in pnts:
            self.addPoint(pnt)
    
    def calculateGaussian(self):
        """
        Compute gaussian for each cell in map.
        """
        grid = self.map_.getGrid()
        for key in grid:
            grid[key].calculateGaussian()
            if grid[key].hasGaussian():
                self.num_of_gaussians = self.num_of_gaussians +1
        
    def getNumberOfGaussians(self):
        """
        Returns the total number of valid Gaussian
        """
        return self.num_of_gaussians
    
    def getMeanAndCov(self):
        means = []
        covs = []
        grid = self.map_.getGrid()
        for key in grid:
            if (grid[key].hasGaussian()):
                means.append(grid[key].getMean())
                covs.append(grid[key].getCovarience())
    
        return np.array(means), np.array(covs)

In [ ]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])

In [ ]:
trg_pcd = load_point_cloud(20, voxel_size=0.005, normalize=0)
trg_rgb_color = np.asarray(trg_pcd.colors)
trg_lab_color = 0.1*rgb2lab(trg_rgb_color)
pnts_trg = np.hstack((np.asarray(trg_pcd.points), trg_lab_color))

trg_map = NDTMap(0.15)
trg_map.addPointCloud(pnts_trg)
trg_map.calculateGaussian()

target_mean, target_cov = trg_map.getMeanAndCov()
target_cov_ = target_cov[:,:3,:3].reshape(-1,9)
target = np.hstack((target_mean, target_cov_))


src_pcd = load_point_cloud(25, voxel_size=0.005, normalize=0)
src_rgb_color = np.asarray(src_pcd.colors)
src_lab_color = 0.1*rgb2lab(src_rgb_color)
pnts_src = np.hstack((np.asarray(src_pcd.points), src_lab_color))

src_map = NDTMap(0.15)
src_map.addPointCloud(pnts_src)
src_map.calculateGaussian()

source_mean, source_cov = src_map.getMeanAndCov()
source_cov_ = source_cov[:,:3,:3].reshape(-1,9)
source = np.hstack((source_mean, source_cov_))

In [ ]:
reg = NDT6DRegistration(source, target)

In [ ]:
for i in indexes:
    trg_pcd = load_point_cloud(i, voxel_size=0.005, normalize=0)
    trg_rgb_color = np.asarray(trg_pcd.colors)
    trg_lab_color = 0.1*rgb2lab(trg_rgb_color)
    pnts_trg = np.hstack((np.asarray(trg_pcd.points), trg_lab_color))

    trg_map = NDTMap(0.15)
    trg_map.addPointCloud(pnts_trg)
    trg_map.calculateGaussian()

    target_mean, target_cov = trg_map.getMeanAndCov()
    target_cov_ = target_cov[:,:3,:3].reshape(-1,9)
    target = np.hstack((target_mean, target_cov_))

    np.savetxt(folder+ "map6d/" + str(i)+'.csv', target, delimiter=',')